In [ ]:
import os
import re
import librosa
import numpy as np
import pandas as pd
import parselmouth
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
AUDIO_DIR = "/content/drive/MyDrive/Dataset/Cleaned_Vocals"
VISUAL_FEATURES_DIR = "/content/drive/MyDrive/Dataset/Visual_Features"
SR = 16000
BATCH_SIZE = 8
EPOCHS = 30
LR = 1e-3
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def set_seed(seed=SEED):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed()

def extract_id_and_label(filename: str):
    match = re.search(r'(stress|nonstress)(\d+)', filename, re.IGNORECASE)
    if match:
        label_str, file_id = match.group(1).lower(), match.group(2)
        return file_id, label_str
    return None, None

In [ ]:
def extract_audio_features(audio_path: str) -> np.ndarray:
    y, sr = librosa.load(audio_path, sr=SR)
    snd = parselmouth.Sound(y, sr)

    try:
        pitch = snd.to_pitch()
        f0 = pitch.selected_array["frequency"]
        f0_mean = float(np.mean(f0[f0 > 0])) if np.any(f0 > 0) else 0.0
    except Exception:
        f0_mean = 0.0

    try:
        point_process = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75, 500)
        jitter = float(parselmouth.praat.call([snd, point_process], "Get jitter (local)", 0, 0, 75, 500, 1.3))
        shimmer = float(parselmouth.praat.call([snd, point_process], "Get shimmer (local)", 0, 0, 75, 500, 1.3, 1.6))
    except Exception:
        jitter, shimmer = 0.0, 0.0

    try:
        hnr = parselmouth.praat.call(snd, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
        hnr_mean = float(parselmouth.praat.call(hnr, "Get mean", 0, 0))
    except Exception:
        hnr_mean = 0.0

    try:
        formant = snd.to_formant_burg()
        f1 = float(parselmouth.praat.call(formant, "Get mean", 1, 0, 0))
        f2 = float(parselmouth.praat.call(formant, "Get mean", 2, 0, 0))
    except Exception:
        f1, f2 = 0.0, 0.0

    try:
        energy = librosa.feature.rms(y=y)[0]
        energy_thresh = 0.02
        voiced = energy > energy_thresh
        speaking_rate = float(np.sum(voiced) / max(1, len(voiced)) * (sr / 512))
    except Exception:
        speaking_rate = 0.0

    return np.array([f0_mean, jitter, shimmer, hnr_mean, f1, f2, speaking_rate], dtype=np.float32)


In [ ]:
class StressDataset(Dataset):
    def __init__(self, audio_dir: str, visual_dir: str):
        self.samples = []
        self.visual_dim = None
        self.logged_once = False

        for subdir in ["stress", "nonstress"]:
            subpath = os.path.join(audio_dir, subdir)
            if not os.path.exists(subpath):
                continue

            for file in os.listdir(subpath):
                if not file.endswith(".wav"):
                    continue

                file_id, label_str = extract_id_and_label(file)
                if file_id is None:
                    continue

                vf_file = os.path.join(visual_dir, f"{label_str}{file_id}_features.csv")
                if not os.path.exists(vf_file):
                    continue

                af = extract_audio_features(os.path.join(subpath, file))
                vf_df = pd.read_csv(vf_file)
                vf_df = vf_df.drop(columns=["timestamp", "video", "label"], errors="ignore")
                vf_df = vf_df.select_dtypes(include=[np.number])

                if not self.logged_once:
                    print(f"✅ Using VF file: {vf_file}")
                    print(f"   Kept {vf_df.shape[1]} numeric columns")
                    self.logged_once = True

                if vf_df.empty:
                    continue

                if self.visual_dim is None:
                    self.visual_dim = vf_df.shape[1]

                vf_seq = vf_df.values.astype(np.float32)
                length = vf_seq.shape[0]
                label = 1 if label_str == "stress" else 0
                self.samples.append((af, vf_seq, length, label))

        if self.visual_dim is None:
            self.visual_dim = 0

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        af, vf_seq, length, label = self.samples[idx]
        return (
            torch.tensor(af, dtype=torch.float32),
            torch.tensor(vf_seq, dtype=torch.float32),
            torch.tensor(length, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )

In [ ]:
def pad_collate(batch):
    af_list, vf_list, len_list, label_list = zip(*batch)
    af_batch = torch.stack(af_list, dim=0)
    lengths = torch.stack(len_list, dim=0)
    labels = torch.stack(label_list, dim=0)

    T_max = max(v.shape[0] for v in vf_list)
    Dv = vf_list[0].shape[1]
    vf_padded = torch.zeros((len(vf_list), T_max, Dv), dtype=torch.float32)
    for i, v in enumerate(vf_list):
        vf_padded[i, :v.shape[0], :] = v

    return af_batch, vf_padded, lengths, labels

In [ ]:
class VisualTemporalCNN(nn.Module):
    def __init__(self, visual_dim: int, hidden_out: int = 128):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=visual_dim, out_channels=256, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(256)
        self.conv2 = nn.Conv1d(in_channels=256, out_channels=hidden_out, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(hidden_out)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()

    def forward(self, vf_padded, lengths):
        B, T, Dv = vf_padded.shape
        if Dv == 0:
            return torch.zeros((B, 128), device=vf_padded.device)
        x = vf_padded.transpose(1, 2)
        x = self.dropout(self.relu(self.bn1(self.conv1(x))))
        x = self.dropout(self.relu(self.bn2(self.conv2(x))))
        mask = torch.arange(T, device=lengths.device).unsqueeze(0) < lengths.unsqueeze(1)
        mask = mask.unsqueeze(1)
        x = x * mask
        sums = x.sum(dim=2)
        counts = mask.sum(dim=2).clamp(min=1)
        return sums / counts


In [ ]:
class FusionModel(nn.Module):
    def __init__(self, audio_dim: int, visual_dim: int, hidden: int = 128):
        super().__init__()
        self.audio_fc = nn.Sequential(
            nn.Linear(audio_dim, hidden),
            nn.ReLU(),
            nn.Dropout(0.3),
        )
        self.visual_temporal = VisualTemporalCNN(visual_dim=visual_dim, hidden_out=hidden)
        self.feature_head = nn.Sequential(
            nn.Linear(hidden * 2, hidden),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden, 2)
        )
        self.alpha = nn.Parameter(torch.tensor(0.5))
        self.audio_head = nn.Linear(hidden, 2)
        self.visual_head = nn.Linear(hidden, 2)

    def forward(self, af, vf_padded, lengths):
        a_h = self.audio_fc(af)
        v_h = self.visual_temporal(vf_padded, lengths)
        feat_logits = self.feature_head(torch.cat([a_h, v_h], dim=1))
        a_logits = self.audio_head(a_h)
        v_logits = self.visual_head(v_h)
        dec_logits = self.alpha * a_logits + (1 - self.alpha) * v_logits
        return (feat_logits + dec_logits) / 2.0



def train_model(dataset, epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR):
    if len(dataset) == 0:
        raise ValueError("Dataset is empty. Check paths.")

    idxs = list(range(len(dataset)))
    train_idx, test_idx = train_test_split(idxs, test_size=0.2, random_state=SEED, shuffle=True,
                                           stratify=[s[3] for s in dataset.samples])

    train_loader = DataLoader(torch.utils.data.Subset(dataset, train_idx),
                              batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
    test_loader = DataLoader(torch.utils.data.Subset(dataset, test_idx),
                             batch_size=batch_size, shuffle=False, collate_fn=pad_collate)

    model = FusionModel(audio_dim=7, visual_dim=dataset.visual_dim).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for af, vf_pad, lengths, labels in train_loader:
            af, vf_pad, lengths, labels = af.to(DEVICE), vf_pad.to(DEVICE), lengths.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            logits = model(af, vf_pad, lengths)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch}/{epochs} - Loss: {total_loss / len(train_loader):.4f}")

    # ==== EVALUATE ====
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for af, vf_pad, lengths, labels in test_loader:
            af, vf_pad, lengths = af.to(DEVICE), vf_pad.to(DEVICE), lengths.to(DEVICE)
            logits = model(af, vf_pad, lengths)
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            y_true.extend(labels.numpy().tolist())
            y_pred.extend(preds.tolist())

    print("Accuracy:", accuracy_score(y_true, y_pred))
    print(classification_report(y_true, y_pred, digits=4))


    save_path = "/content/drive/MyDrive/Stress_Fusion_Model.pth"
    torch.save(model.state_dict(), save_path)
    print(f"✅ Model saved successfully at: {save_path}")
    return model



if __name__ == "__main__":
    dataset = StressDataset(AUDIO_DIR, VISUAL_FEATURES_DIR)
    print(f"Loaded {len(dataset)} samples | Visual feature dim = {dataset.visual_dim}")

    model = train_model(dataset)


    print("\n🔁 Loading saved model for testing...")
    loaded_model = FusionModel(audio_dim=7, visual_dim=dataset.visual_dim)
    loaded_model.load_state_dict(torch.load("/content/drive/MyDrive/Stress_Fusion_Model.pth"))
    loaded_model.to(DEVICE)
    loaded_model.eval()
    print("✅ Model loaded and ready for future testing!")


✅ Using VF file: /content/drive/MyDrive/Dataset/Visual_Features/stress4_features.csv
   Kept 2053 numeric columns
Loaded 807 samples | Visual feature dim = 2053
Epoch 1/30 - Loss: 2.8553
Epoch 2/30 - Loss: 1.3764
Epoch 3/30 - Loss: 0.8788
Epoch 4/30 - Loss: 0.6320
Epoch 5/30 - Loss: 0.5404
Epoch 6/30 - Loss: 0.3722
Epoch 7/30 - Loss: 0.3562
Epoch 8/30 - Loss: 0.2226
Epoch 9/30 - Loss: 0.3353
Epoch 10/30 - Loss: 0.2430
Epoch 11/30 - Loss: 0.1758
Epoch 12/30 - Loss: 0.1966
Epoch 13/30 - Loss: 0.1165
Epoch 14/30 - Loss: 0.1876
Epoch 15/30 - Loss: 0.1405
Epoch 16/30 - Loss: 0.1392
Epoch 17/30 - Loss: 0.0728
Epoch 18/30 - Loss: 0.1124
Epoch 19/30 - Loss: 0.0659
Epoch 20/30 - Loss: 0.0951
Epoch 21/30 - Loss: 0.0676
Epoch 22/30 - Loss: 0.1431
Epoch 23/30 - Loss: 0.1231
Epoch 24/30 - Loss: 0.1019
Epoch 25/30 - Loss: 0.1368
Epoch 26/30 - Loss: 0.0619
Epoch 27/30 - Loss: 0.1045
Epoch 28/30 - Loss: 0.1632
Epoch 29/30 - Loss: 0.0746
Epoch 30/30 - Loss: 0.0331
Accuracy: 0.7901234567901234
         